In [2]:
# Загрузка библиотеки OpenAI-Whisper
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

# Загрузка остальных библиотек
import os
import numpy as np
import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm

try:
    import tensorflow  # требуется в Colab, чтобы избежать проблем с совместимостью
except ImportError:
    pass

# Переключение на GPU, если такое доступно
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Загрузка модели в оперативную память
model = whisper.load_model("large")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters.")

# Настройка опций: временные метки — нужны, язык — русский
options = whisper.DecodingOptions(language="ru", without_timestamps=False)

In [ ]:
import re

# Функция предочистки конспектного текста
def text_preproc(prompt_file):
    file = open(prompt_file, 'r')
    prompt = file.read()
    prompt = prompt.replace("-\n", "").replace("\n", " ")
    prompt = re.sub('[^А-Яа-яЁё\s,.]', '', prompt)
    return prompt

# Текст из конспекта стоит копировать-вставлять постранично
# Не стоит бояться дать слишком много: чем больше исходников, тем лучше и точнее результат
# Вставлять следует текст по смежным темам — корпус слов стоит дать соответствующий

# Очистка и вывод для проверки
prompt_transcribe = text_preproc('prompt.txt')
prompt_transcribe

In [ ]:
# Непосредственно транскрибация
result = model.transcribe("channelL.wav", initial_prompt=prompt_transcribe)

# Экспорт результата в файл
export_file = open("result.txt", "w")
export_file.write(result["text"])
export_file.close()